In [ ]:

from google.colab import files
files.upload() #upload kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"elisaboccini","key":"3024605213579b61d22c8ab4775d25f4"}'}

In [ ]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -q ashirwadsangwan/imdb-dataset
!unzip imdb-dataset.zip

kaggle.json
Archive:  imdb-dataset.zip
  inflating: name.basics.tsv.gz      
  inflating: name.basics.tsv/name.basics.tsv  
  inflating: title.akas.tsv.gz       
  inflating: title.akas.tsv/title.akas.tsv  
  inflating: title.basics.tsv.gz     
  inflating: title.basics.tsv/title.basics.tsv  
  inflating: title.principals.tsv.gz  
  inflating: title.principals.tsv/title.principals.tsv  
  inflating: title.ratings.tsv.gz    
  inflating: title.ratings.tsv/title.ratings.tsv  


In [ ]:
import pandas as pd
principals = pd.read_csv('title.principals.tsv.gz',delimiter="\t")
titlebasics = pd.read_csv('title.basics.tsv.gz',delimiter="\t")
namebasics = pd.read_csv('name.basics.tsv.gz',delimiter="\t")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
cast = principals[principals['category'].isin(['actor', 'actress'])]
movies = titlebasics[titlebasics['titleType'].isin(['movie', 'TVmovie', 'short', 'TVshort'])]

In [ ]:
print(cast.head(2))
print(movies.head(2))

       tconst  ordering     nconst category job      characters
11  tt0000005         1  nm0443482    actor  \N  ["Blacksmith"]
12  tt0000005         2  nm0653042    actor  \N   ["Assistant"]
      tconst titleType  ... runtimeMinutes             genres
0  tt0000001     short  ...              1  Documentary,Short
1  tt0000002     short  ...              5    Animation,Short

[2 rows x 9 columns]


In [ ]:
#taking from principals only tconst, nconst
#taking from titlebasics only tconst and primaryTitle
#taking from namebasics only nconst and primaryName
namebasics = namebasics[['nconst','primaryName']]
print(namebasics.head(2))
movies = movies[['tconst','primaryTitle']]
print(movies.head(2))
cast = cast[['tconst', 'nconst']]
print(cast.head(2))


      nconst    primaryName
0  nm0000001   Fred Astaire
1  nm0000002  Lauren Bacall
      tconst            primaryTitle
0  tt0000001              Carmencita
1  tt0000002  Le clown et ses chiens
       tconst     nconst
11  tt0000005  nm0443482
12  tt0000005  nm0653042


In [ ]:
#merge cast movies namebasic
#first cast and namebasics
actors = cast.merge(namebasics, left_on='nconst', right_on='nconst')
actors.head(5)

,tconst,nconst,primaryName
0,tt0000005,nm0443482,Charles Kayser
1,tt0000005,nm0653042,John Ott
2,tt0000007,nm0179163,James J. Corbett
3,tt0003116,nm0179163,James J. Corbett
4,tt0003730,nm0179163,James J. Corbett


In [ ]:
movies_actors = movies.merge(actors, left_on = 'tconst', right_on='tconst')
movies_actors.head(5)

,tconst,primaryTitle,nconst,primaryName
0,tt0000005,Blacksmith Scene,nm0443482,Charles Kayser
1,tt0000005,Blacksmith Scene,nm0653042,John Ott
2,tt0000007,Corbett and Courtney Before the Kinetograph,nm0179163,James J. Corbett
3,tt0000007,Corbett and Courtney Before the Kinetograph,nm0183947,Peter Courtney
4,tt0000008,Edison Kinetoscopic Record of a Sneeze,nm0653028,Fred Ott


In [ ]:
baskets = pd.DataFrame(movies_actors.groupby(by=['tconst', 'primaryTitle']).apply(lambda x: [x for x in x['primaryName']]))
baskets = baskets.reset_index().rename(columns={0: 'movies_actors'})
print(baskets.head(2))

      tconst  ...                       movies_actors
0  tt0000005  ...          [Charles Kayser, John Ott]
1  tt0000007  ...  [James J. Corbett, Peter Courtney]

[2 rows x 3 columns]


In [ ]:
baskets['movie'] = baskets['tconst'] + '-' + baskets['primaryTitle']

In [ ]:
baskets = baskets[['movie', 'movies_actors']]
baskets.head(2)

,movie,movies_actors
0,tt0000005-Blacksmith Scene,"[Charles Kayser, John Ott]"
1,tt0000007-Corbett and Courtney Before the Kine...,"[James J. Corbett, Peter Courtney]"


In [ ]:
baskets.duplicated('tconst').any()

False

In [ ]:

print(baskets.shape) #887517 movies and cortos

(887517, 3)


In [ ]:
!pip install apyori

  Created wheel for apyori: filename=apyori-1.1.2-cp37-none-any.whl size=5975 sha256=4db43f8f6e1c8c694b6502380e4d305cd5614b3ae17cc6979c545120a57c3af4
  Stored in directory: /root/.cache/pip/wheels/5d/92/bb/474bbadbc8c0062b9eb168f69982a0443263f8ab1711a8cad0
Successfully built apyori


In [ ]:
from sklearn.model_selection import train_test_split
trainAP, testAP = train_test_split(baskets, test_size=0.1)

In [ ]:
!pip install efficient_apriori
from efficient_apriori import apriori

In [ ]:
cast_AP = trainAP['movies_actors'].tolist()
cast_AP= [tuple(r) for r in cast_AP]


In [ ]:
itemsets, rules = apriori(cast_AP, min_support=0.0002, min_confidence=0) # setting same conditions as for fpGrowth to see the differences in output
#I am running Apriori only to the dataset without TVseries (explicit2) BUT i should run it to training set trainMBA2 to actually see how 
#much the results differ
#con 0.0005 it worked but it resulted only 5 frequent itemsets and no association rule, however trying with 0.0001 it takes more than 3 hours
#definitely too slow

In [ ]:
itemsets

In [ ]:
rules #for 0.0003 -> [{Lee Moran} -> {Eddie Lyons}, {Eddie Lyons} -> {Lee Moran}]
#for 0.0001

[{Prem Nazir} -> {Adoor Bhasi},
 {Adoor Bhasi} -> {Prem Nazir},
 {Moe Howard} -> {Larry Fine},
 {Larry Fine} -> {Moe Howard},
 {Jack Richardson} -> {J. Warren Kerrigan},
 {J. Warren Kerrigan} -> {Jack Richardson},
 {Mae Questel} -> {Jack Mercer},
 {Jack Mercer} -> {Mae Questel},
 {Lee Moran} -> {Eddie Lyons},
 {Eddie Lyons} -> {Lee Moran}]